In [4]:
import pandas as pd

In [5]:
# !conda list 

In [6]:
df = pd.read_excel('dataset/Percentuali_di_prelievo_AT_2022-2023.xls')
df = df.rename(columns={"Unnamed: 1": "day_of_week", "Data": "date"})
df['day_of_week'] = pd.to_datetime(df['day_of_week']).dt.day_name()

In [7]:
df

,date,day_of_week,c1%B1,c1%C1,c1%D1,c1%E1,c1%F1,c1%B2,c1%C2,c1%D2,...,c1%B3,c1%C3,c1%D3,c1%E3,c1%F3,c2%,c4%,t1%1,t1%2,t1%3
0,2022-10-01,Saturday,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.087434,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.207365,1.000000e-08,0.277409,3.699878e-01,1.000000e-08
1,2022-10-02,Sunday,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.087434,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.207365,1.000000e-08,0.277409,1.000000e-08,1.000000e-08
2,2022-10-03,Monday,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.109708,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.525225e-01,0.218686,1.000000e-08,0.283256,3.469833e-01,4.267697e-01
3,2022-10-04,Tuesday,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.109708,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.525225e-01,0.218686,1.000000e-08,0.283256,3.469833e-01,4.267697e-01
4,2022-10-05,Wednesday,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.109708,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.525225e-01,0.218686,1.000000e-08,0.283256,3.469833e-01,4.267697e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2023-09-26,Tuesday,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.070471,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,9.993727e-02,0.207365,5.453711e-01,0.286415,3.398387e-01,4.118142e-01
361,2023-09-27,Wednesday,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.070471,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,9.993727e-02,0.207365,5.453711e-01,0.286415,3.398387e-01,4.118142e-01
362,2023-09-28,Thursday,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.070471,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,9.993727e-02,0.207365,5.453711e-01,0.286415,3.398387e-01,4.118142e-01
363,2023-09-29,Friday,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.070471,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,9.993727e-02,0.207365,5.453711e-01,0.286415,3.398387e-01,4.118142e-01


C1 Riscaldamento  
C2 Uso cottura cibi e/o produzione di acqua calda  
C3 Riscaldamento + uso cottura cibi e/o produzione acqua calda sanitaria  
C4 Uso condizionamento  
C5 Uso condizionamento + riscaldamento   

T1 Uso tecnologico (artigianale-industriale)  
T2 Uso tecnologico + riscaldamento  

In [8]:
mapping_name_to_type = {
    "c1": "riscaldamento",
    "c2": "cottura+acqua_calda",
    "c3": "riscaldamento + cottura + acqua calda",
    "c4": "condizionamento",
    "c5": "condizionamento + riscaldamento",
    "t1": "uso tecnologico (artigianale-industriale)",
    "t2": "uso tecnologico + riscaldamento"
}

In [10]:
df.columns.str[:2].unique()

Index(['da', 'c1', 'c2', 'c4', 't1'], dtype='object')

Interpretazione del dataset:
- ogni riga è una giornata relativa al consumo di gas
- ogni colonna è una persona/cliente diverso
- il nome della colonna indica il tipo di consumo di quel cliente secondo il mappaggio sopra definito

In [16]:
(
    df.columns[df.columns.str.contains('c1')].shape, 
    df.columns[df.columns.str.contains('c2')].shape, 
    df.columns[df.columns.str.contains('c4')].shape, 
    df.columns[df.columns.str.contains('t1')].shape
)

((15,), (1,), (1,), (3,))

Abbiamo a disposizione:
- 15 clienti a consumo C1 = solo riscaldamento
- 1 cliente a consumo C2 = cottura + acqua calda
- 1 cliente a consumo C4 = condizionamento
- 3 clienti a consumo T1 = uso tecnologico (artigianale-industriale)"

Non è chiaro se questi siano obbligatoriamente persone diverse, o alcuni di questi profili di consumo appartengono allo stesso.  
Si potrebbe provare a controllare se i profili C2, C4, T1 facciano riferimento ai profili C1 di cui disponiamo.  
Per effettuarlo potremmo controllare come si muovono insieme queste serie rispetto a quelle di C1 con una cross-correlazione.

# Obiettivi analisi descrittiva
0. Quali sono le caratteristiche principali di questo dataset?
1. Quali sono le principali differenze e similitudini fra i diversi profili in termini di stagionalità?
2. Partendo dall’analisi del punto b, a quale tipologia di utilizzo potrebbe realisticamente essere
attribuito ogni profilo?
3. Ci possono essere a tuo parere delle variabili che possono ragionevolmente impattare il
profilo di consumo di un cliente gas che in questi profili non verrebbero considerate?

# Domanda 0